In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

In [2]:
DATASET_PATH = "./DATASET/"
data = pd.read_csv('annotations.csv', index_col=0)
data.head()

,file,bbox,class,size (cm)
0,26_05_21-B20,"[2272.628996958517, 1685.2591150516498, 837.97...",Sepia officinalis,8.435085
1,26_05_21-B25,"[1807.393678506843, 1585.7146585117644, 1094.7...",Mullus barbatus,16.211420
2,26_05_21-B25,"[2648.726235342625, 1368.0723135423739, 345.68...",Mullus barbatus,14.422977
3,26_05_21-B25,"[2845.1096085383197, 1180.704037335252, 426.91...",Mullus barbatus,15.604945
4,26_05_21-B25,"[3192.0335090997633, 1235.4288323372514, 337.1...",Mullus barbatus,14.829357


In [3]:
class DeepFishDataset(Dataset):
    def __init__(self, root_dir: str, data: pd.DataFrame) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.data = data
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
        ])
        self.__filter()

    def __filter(self) -> None:
        self.data.drop(self.data[self.data['class'] != 'Pagrus pagrus'].index, inplace=True)

        deleted_files = []
        for i in range(len(self.data)):
            file = self.data.iloc[i]['file']
            if not self.__is_image_exist(file):
                deleted_files.append(file)
            
        for file in deleted_files:
            self.data.drop(self.data[self.data.file == file].index, inplace=True)       



    def __is_image_exist(self, image_name: str) -> bool:
        return os.path.exists(os.path.join(self.root_dir, f"{image_name}.jpg"))

    def __getitem__(self, index: int):
        image_name = self.__get_image_name(index)
        image = self.__get_image(image_name)
        annotations = self.__get_bbox(image_name)
        bbox = np.zeros((13, 4))
        for i in range(len(annotations)):
            box = annotations[i]
            for c in range(4):
                bbox[i, c] = box[c]
        
        sizes = self.__get_size(image_name)
        size = np.zeros((13, 1))
        for i in range(len(sizes)):
            size[i, 0] = sizes[i]

        if self.transform:
            image = self.transform(image)

        return {
            'image': image,
            'bbox': bbox,
            'size': size,
        }

    def __get_image_name(self, index: int) -> str:
        return self.data.iloc[index]['file']

    def __get_image(self, image_name: str) -> Image.Image:
        return Image.open(os.path.join(self.root_dir, f"{image_name}.jpg")).convert("RGB")

    def __get_bbox(self, image_name: str) -> list:
        strings = self.data[self.data['file'] == image_name]['bbox']
        strings = strings.str[1:-1]
        strings = strings.str.split(',').tolist()
        bbox = [i for i in strings]
        return bbox

    def __get_size(self, image_name: str) -> float:
        return self.data[self.data['file'] == image_name]['size (cm)'].tolist()

    def __len__(self) -> int:
        return len(self.data)
    
dataset = DeepFishDataset(root_dir=DATASET_PATH, data=data)
dataset[5]

{'image': tensor([[[-1.3473, -1.3815, -1.3644,  ..., -1.4158, -1.4158, -1.4329],
          [-1.3644, -1.3987, -1.3987,  ..., -1.3815, -1.3987, -1.4158],
          [-1.3987, -1.3644, -1.3644,  ..., -1.3644, -1.3815, -1.4329],
          ...,
          [-1.2445, -1.1932, -1.2274,  ..., -0.6281, -0.5082, -0.3198],
          [-1.2274, -1.2103, -1.2274,  ..., -0.5253, -0.3369, -0.1828],
          [-1.2103, -1.1932, -1.2617,  ..., -0.5596, -0.1486, -0.2513]],
 
         [[-1.2129, -1.2479, -1.2304,  ...,  0.2927,  0.2752,  0.2577],
          [-1.2304, -1.2654, -1.2654,  ...,  0.2927,  0.2927,  0.2577],
          [-1.3004, -1.2479, -1.2479,  ...,  0.2927,  0.2577,  0.2227],
          ...,
          [-1.2304, -1.1954, -1.2304,  ..., -0.7052, -0.5826, -0.4076],
          [-1.2304, -1.2129, -1.2129,  ..., -0.6001, -0.4426, -0.3025],
          [-1.2129, -1.1954, -1.2654,  ..., -0.6527, -0.2850, -0.3550]],
 
         [[-1.0376, -1.0724, -1.0550,  ...,  1.7511,  1.7511,  1.7337],
          [-1.0724,

In [4]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [5]:
len(dataset), len(train_dataset), len(test_dataset)

(332, 265, 67)

In [7]:
next(iter(train_loader))

{'image': tensor([[[[-1.4329, -1.4843, -1.5014,  ..., -1.0733, -1.0562, -1.0219],
           [-1.4329, -1.4843, -1.5014,  ..., -1.0219, -1.0219, -0.9877],
           [-1.4500, -1.4843, -1.5014,  ..., -0.9192, -0.9363, -0.9534],
           ...,
           [-1.4843, -1.4843, -1.5014,  ..., -0.3369, -0.2856, -0.2856],
           [-1.5014, -1.4672, -1.5185,  ..., -0.3541, -0.3198, -0.3712],
           [-1.5014, -1.4843, -1.4843,  ..., -0.3541, -0.3541, -0.4397]],
 
          [[-1.3354, -1.3704, -1.3704,  ..., -0.9503, -0.9328, -0.8978],
           [-1.3354, -1.3529, -1.3704,  ..., -0.8978, -0.8978, -0.8627],
           [-1.3529, -1.3529, -1.3704,  ..., -0.7927, -0.8102, -0.8277],
           ...,
           [-1.4230, -1.4230, -1.4580,  ..., -0.2500, -0.1975, -0.2150],
           [-1.4405, -1.4055, -1.4580,  ..., -0.2675, -0.2325, -0.2850],
           [-1.4405, -1.4230, -1.4230,  ..., -0.2675, -0.2675, -0.3725]],
 
          [[-1.1073, -1.1421, -1.1596,  ..., -0.8284, -0.8284, -0.8110],
    